In [1]:
import itertools
from collections import Counter
from functools import cache
import numpy as np
from copy import deepcopy

In [2]:
test_input = """Player 1 starting position: 4
Player 2 starting position: 8"""

In [3]:
puzzle_input = open('inputs/21').read().strip()

In [4]:
puzzle_input

'Player 1 starting position: 3\nPlayer 2 starting position: 10'

In [5]:
def parse(s):
    return [int(t.split(': ')[-1]) for t in s.split('\n')]

In [6]:
parse(test_input)

[4, 8]

In [7]:
parse(puzzle_input)

[3, 10]

In [8]:
def p1(puzzle_input):
    player_positions = parse(puzzle_input)
    
    dice_state = 0
    player_scores = [0, 0]
    
    for turn in itertools.count(1):
        player = (turn - 1) % 2

        for _ in range(3):
            dice_state += 1
            player_positions[player] += dice_state

        player_positions[player] = (player_positions[player]-1) % 10 + 1
        player_scores[player] += player_positions[player]

        # print(f"After turn {turn} the player positions are {player_positions}. The scores are {player_scores}")

        if player_scores[player] >= 1000:
            # print(f"Player {player} exceeded 1000. Stopping.")
            break
    
    return min(player_scores) * dice_state

In [9]:
assert p1(test_input) == 739785

In [10]:
p1(puzzle_input)

713328

In [105]:
def p2(puzzle_input):
    player_positions = tuple(parse(puzzle_input))
    player_scores = (0, 0)
    
    WINNING_SCORE = 21
    
    # Reduce 27 options to 7
    possible_dice_sums = Counter(sum(r) for r in itertools.product(range(1, 4), repeat=3)).most_common()
    
    @cache
    def num_wins(current_p_score, current_p_pos, other_p_score, other_p_pos):        
        current_p_wins, other_p_wins = 0, 0

        for summ, count in possible_dice_sums:
            new_position = (current_p_pos + summ - 1) % 10 + 1
            new_score = current_p_score + new_position

            if new_score >= WINNING_SCORE:
                current_p_wins += count
            else:
                subsequent_wins = num_wins(other_p_score, other_p_pos, new_score, new_position)
                current_p_wins += count * subsequent_wins[1]
                other_p_wins += count * subsequent_wins[0]
                
        return current_p_wins, other_p_wins
    
    win_counts = num_wins(player_scores[0], player_positions[0], player_scores[1], player_positions[1])
        
    return max(win_counts)

In [106]:
assert p2(test_input) == 444356092776315

In [107]:
assert p2(puzzle_input) == 92399285032143